In [ ]:
#make sure to build your environment:

#%pip install matplotlib numpy pandas plotly voila 

In [4]:
# ### To run this notebook in "user mode" use:

# In terminal:
# ```voila prototype.ipynb```
# or
# ```voila <path-to-this-notebook>```

# ### This should open your browser and run the notebook without showing the code cells.

In [ ]:
#maybe use streamlit later on, for a cleaner interface

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
# ultimate goal: find a 2D function
# def f(x, y):
#     return ()

# def grad_f(x, y):
#     return ()

X_MIN = -2
X_MAX = 2
# Y_MIN = -2
# Y_MAX = 2

def f(x):
    return x * x + 0.1 * np.sin(10 * x)

def grad_f(x):
    return 2 * x + np.cos(10 * x)

def gradient_descent(a_0, eta, max_iter):
    a_n = a_0
    a_ns = np.zeros(max_iter)
    a_ns[0] = a_0
    losses = np.zeros(max_iter)
    losses[0] = compute_loss(a_0, f)
    for i in range(1, max_iter):
        a_n_1 = a_n - eta * grad_f(a_n)
        a_ns[i] = a_n_1
        losses[i] = compute_loss(a_n_1, f)
        a_n = a_n_1
    return a_n_1, losses[-1], a_ns, losses

def compute_loss(a_final, f):
    return abs(a_final - find_min(f))

def find_min(f):
    # 1D for now but should be 2D later on
    x = np.linspace(X_MIN, X_MAX, 250)
    true_min = min(f(x))
    return true_min



In [8]:
a_0 = -1.5
eta = 0.1
max_iter = 15
a_final, final_loss, a_ns, losses = gradient_descent(a_0, eta, max_iter)
df_gd = pd.DataFrame({'a_ns': a_ns, 'f_a_ns': f(a_ns), 'losses': losses, 'iteration': np.arange(max_iter)})

In [9]:
def plot_iterations(df_gd, X_MIN, X_MAX):

    # Create base figure with the function line
    x = np.linspace(X_MIN, X_MAX, 300)
    fig = go.Figure()

    # Add the function line (will stay static in background)
    fig.add_trace(
        go.Scatter(
            x=x,
            y=f(x),
            mode='lines',
            name='f(x)',
            line=dict(color='lightgray'),
        )
    )

    # Add the animated scatter points
    fig.add_trace(
        go.Scatter(
            x=[df_gd['a_ns'][0]],
            y=[df_gd['f_a_ns'][0]],
            mode='markers',
            name='Gradient Descent',
            marker=dict(
                color='red',
                size=7
            )
        )
    )

    # Add animation settings
    fig.update_layout(
        updatemenus=[{
            'type': 'buttons',
            'showactive': False,
            'buttons': [{
                'label': 'Play',
                'method': 'animate',
                'args': [None, {
                    'frame': {'duration': 600, 'redraw': True},
                    'fromcurrent': True,
                    'transition': {'duration': 200}
                }]
            },
                {
                    'label': 'Pause',
                    'method': 'animate',
                    'args': [[None], {
                        'frame': {'duration': 0, 'redraw': False},
                        'mode': 'immediate',
                        'transition': {'duration': 0}
                    }]
                }
            ]
        }],
        sliders=[{
            'steps': [
                {
                    'args': [[str(i)], {'frame': {'duration': 0, 'redraw': True},
                                        'mode': 'immediate'}],
                    'label': str(i),
                    'method': 'animate'
                }
                for i in df_gd['iteration'].unique()
            ],
            'transition': {'duration': 200},
            'x': 0,
            'y': -0.2,
            'currentvalue': {'prefix': 'Iteration: '}
        }]
    )

    # Create frames for animation
    frames = [
        go.Frame(
            data=[
                # Keep the line trace unchanged
                go.Scatter(x=x, y=f(x)),
                # Update only the scatter points
                go.Scatter(
                    x=df_gd[df_gd['iteration'] <= i]['a_ns'],
                    y=df_gd[df_gd['iteration'] <= i]['f_a_ns']
                )
            ],
            name=str(i)
        )
        for i in df_gd['iteration'].unique()
    ]

    fig.frames = frames

    # Update layout
    fig.update_layout(
        height = 700,
        #width = 800,
        xaxis_range=[X_MIN, X_MAX],
        yaxis_range=[min(f(x)), max(f(x))],
        title='Gradient Descent Animation',
        showlegend=True
    )

    fig.show()

In [10]:
#plot_iterations(df_gd, X_MIN, X_MAX)

In [11]:
def plot_loss(df_gd, max_iter):
    # Create base figure
    fig = go.Figure()

    # Add an empty trace (to be animated later)
    fig.add_trace(
        go.Scatter(
            x=[],
            y=[],
            mode="markers+lines",
            marker=dict(color="red", size=7),
            name="Loss"
        )
    )

    # Create frames for animation
    frames = []
    for i in df_gd['iteration'].unique():
        frames.append(
            go.Frame(
                data=[
                    # leave line untouched → only scatter trace (2nd trace, index 1)
                    go.Scatter(
                        x=df_gd.loc[df_gd['iteration'] <= i, 'iteration'],
                        y=df_gd.loc[df_gd['iteration'] <= i, 'losses'],
                        mode="markers+lines",
                        marker=dict(color='red', size=7)
                    )
                ],
                name=str(i)
            )
        )

    fig.frames = frames

    # Add animation controls
    fig.update_layout(
        height = 700,
        xaxis_range=[-1, max_iter],
        yaxis_range=[-0.1, max(df_gd.losses) + 0.1],
        title='Gradient Descent Loss',
        showlegend=True,
        updatemenus=[{
            'type': 'buttons',
            'showactive': False,
            'buttons': [
                {
                    'label': 'Play',
                    'method': 'animate',
                    'args': [None, {
                        'frame': {'duration': 600, 'redraw': True},
                        'fromcurrent': True,
                        'transition': {'duration': 200}
                    }]
                },
                {
                    'label': 'Pause',
                    'method': 'animate',
                    'args': [[None], {
                        'frame': {'duration': 0, 'redraw': False},
                        'mode': 'immediate',
                        'transition': {'duration': 0}
                    }]
                }
            ]
        }],
        sliders=[{
            'steps': [
                {
                    'args': [[str(i)], {'frame': {'duration': 0, 'redraw': True},
                                        'mode': 'immediate'}],
                    'label': str(i),
                    'method': 'animate'
                }
                for i in df_gd['iteration'].unique()
            ],
            'transition': {'duration': 200},
            'x': 0,
            'y': -0.2,
            'currentvalue': {'prefix': 'Iteration: '}
        }]
    )

    fig.show()


In [12]:
#plot_loss(df_gd, max_iter)

In [13]:
def plot_iterations_and_loss(df_gd, X_MIN, X_MAX, max_iter):
    # Create subplot layout (1 row, 2 columns)
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Gradient Descent Path", "Loss Curve"))

    # ===== Left subplot: f(x) + GD path =====
    x = np.linspace(X_MIN, X_MAX, 300)
    fig.add_trace(
        go.Scatter(
            x=x,
            y=f(x),
            mode="lines",
            name="f(x)",
            line=dict(color="lightgray"),
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=[df_gd['a_ns'].iloc[0]],
            y=[df_gd['f_a_ns'].iloc[0]],
            mode="markers",
            marker=dict(color="red", size=7),
            name="GD Path"
        ),
        row=1, col=1
    )

    # ===== Right subplot: Loss curve =====
    fig.add_trace(
        go.Scatter(
            x=[],
            y=[],
            mode="markers+lines",
            marker=dict(color="red", size=7),
            name="Loss"
        ),
        row=1, col=2
    )

    # ===== Frames: update both subplots together =====
    frames = []
    for i in df_gd['iteration'].unique():
        frames.append(
            go.Frame(
                data=[
                    # Left: 
                    # Keep the line trace unchanged
                    go.Scatter(x=x, y=f(x)),
                    # Then update GD points
                    go.Scatter(
                        x=df_gd.loc[df_gd['iteration'] <= i, 'a_ns'],
                        y=df_gd.loc[df_gd['iteration'] <= i, 'f_a_ns'],
                        marker=dict(color="red", size=7)
                    ),
                    # Right: update loss curve
                    go.Scatter(
                        x=df_gd.loc[df_gd['iteration'] <= i, 'iteration'],
                        y=df_gd.loc[df_gd['iteration'] <= i, 'losses'],
                        mode="markers+lines",
                        marker=dict(color="red", size=7)
                    )
                ],
                name=str(i)
            )
        )

    fig.frames = frames

    # ===== Layout with shared controls =====
    fig.update_layout(
        height=700,
        #width=1000,
        title="Gradient Descent Animation",
        xaxis=dict(range=[X_MIN - 0.2, X_MAX + 0.2], title="x"),
        yaxis=dict(range=[min(f(x)) - 0.3, max(f(x)) + 0.3], title="f(x)"),
        xaxis2=dict(range=[-1, max_iter], title="Iteration"),
        yaxis2=dict(range=[-0.1, max(df_gd['losses']) + 0.1], title="Loss"),
        showlegend=False,
        updatemenus=[{
            'type': 'buttons',
            'showactive': False,
            'buttons': [
                {
                    'label': 'Play',
                    'method': 'animate',
                    'args': [None, {
                        'frame': {'duration': 600, 'redraw': True},
                        'fromcurrent': True,
                        'transition': {'duration': 200}
                    }]
                },
                {
                    'label': 'Pause',
                    'method': 'animate',
                    'args': [[None], {
                        'frame': {'duration': 0, 'redraw': False},
                        'mode': 'immediate',
                        'transition': {'duration': 0}
                    }]
                }
            ]
        }],
        sliders=[{
            'steps': [
                {
                    'args': [[str(i)], {
                        'frame': {'duration': 0, 'redraw': True},
                        'mode': 'immediate'
                    }],
                    'label': str(i),
                    'method': 'animate'
                }
                for i in df_gd['iteration'].unique()
            ],
            'transition': {'duration': 200},
            'x': 0,
            'y': -0.2,
            'currentvalue': {'prefix': 'Iteration: '}
        }]
    )

    fig.show()


In [14]:
plot_iterations_and_loss(df_gd, X_MIN, X_MAX, max_iter)